# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [9]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [50]:
df


,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to l...
5,L'Or Espresso Café,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)

#chosen temperature is 0.7 because it is a good balance between creativity and coherence


In [14]:
prompt = ChatPromptTemplate.from_template( 
    """Make a 2 to 3 sentence description about this product
    product: {product}"""
)

In [15]:

chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/2f/9101t8nx1hn47xc4fzmkb13w0000gn/T/ipykernel_9262/546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [30]:
product = df.iloc[3]['Product']
ai_response = chain.run(product)
print(f" Product: {product} \n AI Response: {ai_response}")

 Product: Pillows Insert 
 AI Response: Our pillow inserts are the perfect addition to any decorative pillow cover, providing extra support and comfort for a good night's sleep or cozy lounging. Made with high-quality materials and available in various sizes to fit your pillow covers perfectly. Upgrade your home decor with our plush and durable pillow inserts.


## SimpleSequentialChain

In [32]:
from langchain.chains import SimpleSequentialChain

In [42]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    """Make a long detailed  description about this product
    product: {product}"""
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [43]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    """Write a bullet point list of pros and cons from the description of the product.
    description: {description}"""
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [44]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [45]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Pillow inserts are an essential component of any bedding set, providing support and comfort for a good night's sleep. These inserts are designed to fit snugly inside pillow covers, ensuring a plump and luxurious appearance. 

These pillow inserts are made from high-quality materials that are both soft and durable. The outer material is usually a smooth and soft fabric such as cotton or microfiber, while the filling is often a hypoallergenic material like polyester fiberfill. This ensures that the pillow inserts are gentle on the skin and safe for those with allergies.

The construction of these pillow inserts is also top-notch, with double-stitched seams for added durability. This means that the inserts will hold their shape well over time, maintaining their plushness and support. The size of the inserts can vary to fit different pillow covers, ranging from standard to king size.

One of the key benefits of using pillow inserts is their v

"Pros:\n- Provides support and comfort for a good night's sleep\n- Designed to fit snugly inside pillow covers for a luxurious appearance\n- Made from high-quality, soft, and durable materials\n- Hypoallergenic filling is gentle on the skin and safe for those with allergies\n- Double-stitched seams for added durability\n- Versatile and can easily be swapped out for different pillow covers\n- Easy to care for, machine washable and tumble dry on low heat\n\nCons:\n- May need to be replaced over time as they may lose their plushness and support\n- Different sizes may be needed for different pillow covers, adding to cost\n- Not all pillow inserts may be machine washable, requiring special care instructions for some."

**Repeat the above twice for different products**

## SequentialChain

In [48]:
from langchain.chains import SequentialChain

In [53]:
llm = ChatOpenAI(temperature=0.9)
review = df.iloc[5]['Review']
print(review)

first_prompt = ChatPromptTemplate.from_template(
  """Please provide a spanish translation to the following review
  review: {review}"""
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="spanish" #Give a name to your output
                    )


Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...Vieux lot ou contrefaçon !?


In [54]:
second_prompt = ChatPromptTemplate.from_template(
     """Please provide a summary of the following review
        review: {spanish}"""
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [55]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
      """Please provide a english translation to the following review
        review: {summary}"""
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="english"
                      )


In [56]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables

fourth_prompt = ChatPromptTemplate.from_template(
        "Based on the english translation, summary, craft a follow-up message:\n"
        "english Translation: {english}\n"
        "Summary: {summary}\n"
        "Follow-up Message:"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [ ]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english", "summary", "followup_message"],
    verbose=True
)


In [62]:
review = df.Review[5]
results = overall_chain(review)
results



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...Vieux lot ou contrefaçon !?",
 'english': "Translation: The reviewer believes the taste is average and points out that the foam doesn't last long, which they find odd. They mention that they bought the same product from a store and found the taste to be significantly better, making them wonder if the product they have is from an older batch or if it's a fake.",
 'summary': 'The reviewer finds the flavor mediocre and notes that the foam does not hold well, which they find strange. They mention that they have purchased the same product in a store and found the flavor to be much better, leading them to question if the product they have is from an old batch or if it is a counterfeit.',
 'followup_message': 'Dear valued customer,\n\nWe are sorry to hear about your experience with our product. We take quality very seriously and would like to invest

## Router Chain

In [63]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [64]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [65]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [66]:
llm = ChatOpenAI(temperature=0)

In [67]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [68]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [69]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [70]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [71]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/2f/9101t8nx1hn47xc4fzmkb13w0000gn/T/ipykernel_9262/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [72]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [73]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'